In [2]:
import os
import sys

import numpy as np
import pandas as pd
import seaborn as sns

import skombo
from loguru import logger as log


from pandas.io.formats.style import Styler
from skombo import TEST_COMBO_CSVS
from skombo.combo_calc import ComboCalculator
from skombo.fd_ops import FD_COLS, FD, FD_BOT_CSV_MANAGER

pd.set_option("display.max_columns", 20)
pd.set_option("display.max_colwidth", 20)
pd.set_option("display.width", 100)

In [3]:
def color_code(col: pd.Series, pal: str):
    try:
        unique = col.loc[col.apply(pd.notna)].unique()
    except TypeError:
        # handle case where col is a series of lists by flattening each list into a string
        col = col.apply(lambda x: ",".join(x) if isinstance(x, list) else x)
        try:
            unique = col.loc[col.apply(pd.notna)].unique()
        except TypeError:
            # handle case where col is a series of dicts by flattening each dict into a string
            col = col.apply(
                lambda x: ",".join(":".join([str(k), str(v)]) for k, v in x.items())
                if isinstance(x, dict)
                else x
            )
            unique = col.loc[col.apply(pd.notna)].unique()

    palette = sns.color_palette(pal, len(unique))
    # dict of {unique value: colour}
    colour_map = {unique[i]: palette[i] for i in range(len(unique))}  # type: ignore
    # Seperate dict for text colour, invert the hex values for now
    text_colour_map = dict(colour_map)
    # r,g,b is a tuple of floats
    for k, v in text_colour_map.items():
        r, g, b = v
        luma = 0.2126 * r + 0.7152 * g + 0.0722 * b  # type: ignore
        text_colour_map[k] = "#ffffff" if luma < 0.5 else "#000000"

    # convert to hex
    colour_map = {
        # type: ignore
        k: sns.palettes.color_palette([v]).as_hex()[0] # type: ignore
        for k, v in colour_map.items()
    }
    text_colour_map = {
        k: sns.palettes.color_palette([v]).as_hex()[0]  # type: ignore
        for k, v in text_colour_map.items()
    }

    def color_cell(val):
        return (
            f"background-color: {colour_map[val]} ; color: {text_colour_map[val]}"
            if pd.notna(val)
            else ""
        )

    return col.apply(color_cell)

In [4]:
test_import = ComboCalculator(TEST_COMBO_CSVS[0])

display(test_import.input_combos.style.apply(color_code, pal="Set2")) # type: ignore
test_import.input_combos.dtypes

🌟|INFO    | Loaded 3 combos from C:\Users\micah\repos\skugs\tests\test_data\annie_test_combos.csv


,name,character,assist_1,assist_2,own_team_size,opponent_team_size,notation,counter_hit,undizzy,expected_damage,meter
0,ANNIE_0,ANNIE,umbrella,nan,3,3,2HP otg M crescent 5mp 5hpx2 kara divekick 66 jhp launcher jc jlk jhp adc jlp jmp 5lk 2mp 5mkx2 5hp h knuckle,False,0,0,nan
1,ANNIE_0,ANNIE,umbrella,nan,3,3,2HP otg M crescent 5mp 5hpx2 kara divekick 66 jhp launcher jc jlk jhp adc jlp jmp 5lk 5mp 5mkx2 5hp h knuckle,False,0,0,nan
2,ANNIE_0,ANNIE,umbrella,nan,3,3,2HP otg M crescent 5mp 5hpx2 kara divekick 66 jhp launcher jc jlk jhp adc jlp jmp 5lpx2 5mp 5mkx2 5hp h knuckle,False,0,0,nan


name                  string[python]
character             string[python]
assist_1              string[python]
assist_2                      object
own_team_size                   Int8
opponent_team_size              Int8
notation              string[python]
counter_hit                  boolean
undizzy                         Int8
expected_damage                Int16
meter                         object
dtype: object

In [5]:
# Filter out the columns we don't want to display
remove_cols = [FD_COLS.a_names, FD_COLS.footer, FD_COLS.footer_url, FD_COLS.thumb_url]
cols_to_display = [col for col in FD.columns if col not in remove_cols]


def df_snapshot(df, rows=30):
    displaydf = df.head(rows)
    displaydf = pd.concat(
        [
            displaydf,
            pd.DataFrame(
                data=[[np.nan] * len(displaydf.columns)],
                columns=displaydf.columns,
            ),
        ]
    )
    displaydf = pd.concat([displaydf, df.tail(rows)])

    return displaydf


df_snapshot(FD[cols_to_display]).style.apply(color_code, pal="Set2") # type: ignore

c:\Users\micah\repos\skugs\.venv\Lib\site-packages\pandas\core\dtypes\missing.py:557: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return lib.array_equivalent_object(ensure_object(left), ensure_object(right))
c:\Users\micah\repos\skugs\.venv\Lib\site-packages\pandas\core\dtypes\missing.py:557: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return lib.array_equivalent_object(ensure_object(left), ensure_object(right))
c:\Users\micah\repos\skugs\.venv\Lib\site-packages\pandas\core\dtypes\missing.py:557: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return lib.array_equivalent_object(ensure_object(left), ensure_object(right))
c:\Users\micah\repos\skugs\.venv\Lib\site-packages\pandas\core\dtypes\missing.py:557: FutureWarning: elementwise comparison 

,guard,properties,damage,chip_damage,meter_on_hit,meter_on_whiff,on_hit_adv,on_hit_effect,on_block_adv,startup,active,recovery,hitstun,blockstun,hitstop,blockstop,super_hitstop,on_pushblock,move_category,undizzy,scaling_effect,scaling_for_hit,scaling_after_modifiers
"('ANNIE', '5LP')",['Mid'],['Chains Into Self'],['300'],nan,['2.50'],nan,1,nan,4,['6'],['2'],['14'],['16'],['19'],['7'],nan,nan,"['11', '12']",LIGHT_NORMAL,15.000000,nan,nan,nan
"('ANNIE', '5LP X2')",['Mid'],['-'],['300'],nan,['2.50'],nan,1,nan,4,"['11 (whiff)', ' 6 (hit/block)']",['2'],['14'],['16'],['19'],['7'],nan,nan,"['11', '12']",LIGHT_NORMAL,15.000000,nan,nan,nan
"('ANNIE', '5MP')",['Mid'],['-'],['500'],nan,['7.50'],nan,-2,nan,-11,['12'],['2'],['26'],['25'],['16'],['9'],nan,nan,"['-1', '0']",MEDIUM_NORMAL,20.000000,nan,nan,nan
"('ANNIE', '5HP')",['Mid'],['Chains Into Self'],['700'],nan,['4.5'],nan,0,nan,-11,['16'],['2'],['33'],['34'],['23'],['11'],nan,nan,"['-8', '-7']",HEAVY_NORMAL,30.000000,nan,nan,nan
"('ANNIE', '5HP X2')",['High'],['-'],['600'],nan,['4.5'],nan,nan,Soft KD,-6,['19'],['2'],['21'],['27'],['16'],['11'],nan,nan,"['4', '5']",HEAVY_NORMAL,30.000000,nan,nan,nan
"('ANNIE', '5LK')",['Mid'],['-'],"['200', '200', '300']",nan,"['2.7', '2.7', '3.15']",nan,6,nan,0,['8'],"['4', '4', '4']",['15'],"['25', '25', '24']","['17', '17', '18']","['4', '4', '9']",nan,nan,"['0', '8']",LIGHT_NORMAL,15.000000,nan,nan,nan
"('ANNIE', '5MK')",['Mid'],['Chains Into Self'],['400'],nan,['3.75'],nan,3,nan,3,['11'],['4'],['17'],['23'],['23'],['10'],nan,nan,"['6', '9']",MEDIUM_NORMAL,20.000000,nan,nan,nan
"('ANNIE', '5MK X2')",['Mid'],['-'],['500'],nan,['7.5'],nan,-2,nan,-7,['10'],['3'],['24'],['24'],['19'],['9'],nan,nan,"['0', '2']",MEDIUM_NORMAL,20.000000,nan,nan,nan
"('ANNIE', '5HK')",['Mid'],['Launcher'],['950'],nan,['10'],nan,-3,nan,-14,['17'],['6'],['28'],['30'],['19'],['14'],['10'],nan,"['-11', '-6']",HEAVY_NORMAL,30.000000,nan,nan,nan
"('ANNIE', '2LP')",['Mid'],['Chains Into Self'],['300'],nan,['2.5'],nan,2,nan,1,['6'],['3'],['15'],['19'],['18'],['7'],nan,nan,"['9', '26']",LIGHT_NORMAL,15.000000,nan,nan,nan
